**SHAKTI SINGH (2311MC13)**

**NLP Assignment 02**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import math

In [ ]:
df = pd.read_csv("NER-Dataset-Train.csv")

In [ ]:
df.head()

,Sentences with BIO tags
0,@LewisDixon O Trust O me O ! O im O gonna O be...
1,@joshHnumber1fan O its O okay O then O .. O ma...
2,"Asprin O , O check O , O cup O of O tea O , O ..."
3,@angelportugues O LMAO O ! O When O is O tht O...
4,The O Basic O Step O Before O You O Even O Sta...


In [ ]:
# Creating trasition matrix

def create_transition_matrix(X_train):
    #declaring transition matrix
    transition_count_t1_t2 = [[float(0) for row in range(3)] for col in range(3)]
    BIO_tags = {'B':0,'I':1,'O':2}
    count_of_each_BIO_tags = [0]*3
    BIO_tags_as_initial_count = [0]*3
    count  = 0
    for i in X_train:
        line = list(i.split(" "))
        BIO_tags_as_initial_count[BIO_tags[line[1]]]+=1
        count_of_each_BIO_tags[BIO_tags[line[1]]]+=1
        length  = int(len(line))
        for j in range(3,length,2):
            count_of_each_BIO_tags[BIO_tags[line[j]]]+=1
            transition_count_t1_t2[BIO_tags[line[j-2]]][BIO_tags[line[j]]] += 1

    return transition_count_t1_t2,BIO_tags_as_initial_count,count_of_each_BIO_tags

In [ ]:
def create_transition_matrix_trigram(X_train):
    # Declaring transition matrix
    transition_count_t1_t2 = [[[float(0) for inner_row in range(3)] for row in range(3)] for col in range(3)]
    BIO_tags = {'B': 0, 'I': 1, 'O': 2}
    count_of_each_BIO_tags = [0] * 3
    BIO_tags_as_initial_count = [0] * 3
    count = 0
    for i in X_train:
        line = list(i.split(" "))
        BIO_tags_as_initial_count[BIO_tags[line[1]]] += 1
        count_of_each_BIO_tags[BIO_tags[line[1]]] += 1
        length = int(len(line))
        for j in range(5, length, 2):  # Starting from 5 for trigram processing
            count_of_each_BIO_tags[BIO_tags[line[j]]] += 1
            transition_count_t1_t2[BIO_tags[line[j-4]]][BIO_tags[line[j-2]]][BIO_tags[line[j]]] += 1

    return transition_count_t1_t2, BIO_tags_as_initial_count, count_of_each_BIO_tags

In [ ]:
# Calculating start probability

def calculate_start_probability(BIO_tags_as_initial_count,count_of_each_BIO_tags):
    start_probability = [0]*3
    for i in range(3):
        start_probability[i] = (BIO_tags_as_initial_count[i])/count_of_each_BIO_tags[i]
    return start_probability

In [ ]:
# Calculate transition probability for trigram model

def calculate_transition_probability_trigram(X_train):
    transition_count_t1_t2_t3, BIO_tags_as_initial_count, count_of_each_BIO_tags = create_transition_matrix_trigram(X_train)
    calculate_start_probability(BIO_tags_as_initial_count, count_of_each_BIO_tags)

    # Initialize transition_probability_t1_t2_t3 as a 3-dimensional list
    #transition_probability_t1_t2_t3 = [[[0.0 for _ in range(3)] for _ in range(3)] for _ in range(3)]
    transition_probability_t1_t2_t3 = copy.deepcopy(transition_count_t1_t2_t3)


    for i in range(3):  # Assuming 3 BIO tags, you may adjust this range according to your data
        for j in range(3):
            for k in range(3):
                # Calculate transition probabilities
                transition_probability_t1_t2_t3[i][j][k] = (transition_count_t1_t2_t3[i][j][k]) / (count_of_each_BIO_tags[i] + 1)

    return transition_probability_t1_t2_t3, count_of_each_BIO_tags, BIO_tags_as_initial_count

In [ ]:
# Calculate transition probability for bigram model

def calculate_transition_probability_bigram(X_train):
    transition_count_t1_t2,BIO_tags_as_initial_count,count_of_each_BIO_tags = create_transition_matrix(X_train)
    calculate_start_probability(BIO_tags_as_initial_count,count_of_each_BIO_tags)

    transition_probability_t1_t2 = copy.deepcopy(transition_count_t1_t2)

    for i in range(0,3):
        for j in range(0,3):
            transition_probability_t1_t2[i][j] = (transition_probability_t1_t2[i][j])/(count_of_each_BIO_tags[i] + 1)

    return transition_probability_t1_t2,count_of_each_BIO_tags,BIO_tags_as_initial_count

In [ ]:
def extracting_distinct_words(X_train):
    distinct_words = set()
    i = 0
    for i in X_train:
        line = list(i.split(" "))
        length  = int(len(line))
        for j in range(0,length,2):
            distinct_words.add(line[j])

    words = {}
    j = 0
    for i in distinct_words:
        words[i] = j
        j = j + 1
    return words

In [ ]:
def calculate_emission_count(X_train):
    words = extracting_distinct_words(X_train)
    BIO_tags = {'B':0,'I':1,'O':2}

    count_of_distinct_words = len(words)
    #declaring emission count matrix
    emission_count = []
    for i in range(0,count_of_distinct_words):
        t = []
        for j in range(0,3):
            t.append(float(0))
        emission_count.append(t)

    for i in range(0,len(X_train)):
        linex = X_train[i].split(" ")
        length = len(linex)

        for j in range(0,length,2):
            emission_count[words[linex[j]]][BIO_tags[linex[j+1]]] += 1
    return emission_count,count_of_distinct_words,words

In [ ]:
def calculate_emission_probability(X_train,count_of_each_BIO_tags):
    emission_count,count_of_distinct_words,words = calculate_emission_count(X_train)
    emission_probability = copy.deepcopy(emission_count)

    for i in range(0,count_of_distinct_words):
        for j in range(0,3):
            emission_probability[i][j] = (emission_probability[i][j])/(count_of_each_BIO_tags[j])

    return emission_probability,words

In [ ]:
def most_frequent_BIO_tags(count_of_each_BIO_tags):
    BIO_tags = {'B':0,'I':1,'O':2}
    for i in range(0,3):
        if(count_of_each_BIO_tags[i]==max(count_of_each_BIO_tags)):
            most_frequent_tag_index = i
            break

    for i in BIO_tags:
        if(BIO_tags[i]==most_frequent_tag_index):
            most_frequent_tag = i
    return most_frequent_tag

In [ ]:
def viterbiAlgorithm_Bigram(transition_probability,emission_probability,start_probability,count_of_each_pos_tags,pos_tags,words,most_frequent_tag,testX):
    count_of_distinct_pos_tags = len(pos_tags)
    BIO_tags = {'B':0,'I':1,'O':2}

    #declaring matrix to calculate probability at each k
    prob_and_seq = []
    cursen = testX.split(" ")
    curlen = len(cursen)
    cursenlen = int(curlen/2)
    predicted_seq_of_pos_tags = []

    for i in range(0,count_of_distinct_pos_tags):
        t = []
        for j in range(0,cursenlen):
            t.append([0,"O"])
        prob_and_seq.append(t)
    #end of matrix creation

    for j in pos_tags.keys():
        if cursen[0] in words.keys():
            prob_and_seq[pos_tags[j]][0][0] = start_probability[pos_tags[j]] * emission_probability[words[cursen[0]]][pos_tags[j]]
            prob_and_seq[pos_tags[j]][0][1] = pos_tags[j]

        else:
            prob_and_seq[pos_tags[j]][0][0] = start_probability[pos_tags[j]]
            prob_and_seq[pos_tags[j]][0][1] = pos_tags[j]



    for i in range(1,cursenlen):
        if cursen[i*2] in words.keys():
            for j in pos_tags.keys(): #every prev state
                prev_prob = prob_and_seq[pos_tags[j]][i-1][0]

                for k in pos_tags.keys():#every cur state
                    max_prob = prob_and_seq[pos_tags[k]][i][0]
                    max_postag = prob_and_seq[pos_tags[k]][i][1]

                    p1 = transition_probability[pos_tags[j]][pos_tags[k]]
                    p2 = emission_probability[words[cursen[i*2]]][pos_tags[k]]

                    cur_prob = prev_prob + math.log(p1 + 1) + math.log(p2 + 1)

                    if max_prob<=cur_prob:
                            max_prob = cur_prob
                            prob_and_seq[pos_tags[k]][i][0] = max_prob
                            prob_and_seq[pos_tags[k]][i][1] = j

        else:
            for j in pos_tags.keys(): #every prev state
                prev_prob = prob_and_seq[pos_tags[j]][i-1][0]

                for k in pos_tags.keys():#every cur state
                    if (k==most_frequent_tag):
                        max_prob = prob_and_seq[pos_tags[k]][i][0]
                        max_postag = prob_and_seq[pos_tags[k]][i][1]

                        p1 = transition_probability[pos_tags[j]][pos_tags[k]]

                        cur_prob = prev_prob + math.log(p1 + 1)

                        if max_prob<=cur_prob:
                                max_prob = cur_prob
                                prob_and_seq[pos_tags[k]][i][0] = max_prob
                                prob_and_seq[pos_tags[k]][i][1] = j
                    else:
                        prob_and_seq[pos_tags[k]][i][0] = float(0)
                        prob_and_seq[pos_tags[k]][i][1] = "O"

            #print(prob_and_seq[pos_tags[k]][i][1])

    maxiprob = prob_and_seq[0][cursenlen-1][0]
    prevpostag = prob_and_seq[0][cursenlen-1][1]

    for i in pos_tags.keys():
        if prob_and_seq[pos_tags[i]][cursenlen-1][0]>=maxiprob:
            maxiprob = prob_and_seq[pos_tags[i]][cursenlen-1][0]
            prevpostag = prob_and_seq[pos_tags[i]][cursenlen-1][1]
            maxipostag = i


    predicted_seq_of_pos_tags.append(maxipostag)

    for i in range(cursenlen-2,-1,-1):
        predicted_seq_of_pos_tags.append(prevpostag)
        prevpostag = prob_and_seq[pos_tags[prevpostag]][i][1]


    predicted_seq_of_pos_tags.reverse()

    #print(predicted_seq_of_pos_tags)
    return predicted_seq_of_pos_tags

In [ ]:
import math

def viterbiAlgorithm_Trigram(transition_probability, emission_probability, start_probability, count_of_each_pos_tags, pos_tags, words, most_frequent_tag, testX):
    count_of_distinct_pos_tags = len(pos_tags)

    # Declaring matrix to calculate probability at each k
    prob_and_seq = []
    cursen = testX.split(" ")
    curlen = len(cursen)
    cursenlen = int(curlen / 2)
    predicted_seq_of_pos_tags = []

    for i in range(0, count_of_distinct_pos_tags):
        t = []
        for j in range(0, cursenlen):
            t.append([0, "O"])
        prob_and_seq.append(t)

    # Start with the first word
    for j in pos_tags.keys():
        if cursen[0] in words.keys():
            prob_and_seq[pos_tags[j]][0][0] = start_probability[pos_tags[j]] * emission_probability[words[cursen[0]]][pos_tags[j]]
            prob_and_seq[pos_tags[j]][0][1] = pos_tags[j]
        else:
            prob_and_seq[pos_tags[j]][0][0] = start_probability[pos_tags[j]]
            prob_and_seq[pos_tags[j]][0][1] = pos_tags[j]

    # Iterate through the rest of the words
    for i in range(1, cursenlen):
        if cursen[i * 2] in words.keys():
            for k in pos_tags.keys():  # current state
                max_prob = float('-inf')
                max_postag = ""

                for j in pos_tags.keys():  # previous state
                    for l in pos_tags.keys():  # previous previous state
                        prev_prob = prob_and_seq[pos_tags[j]][i - 1][0]
                        prev_prev_prob = prob_and_seq[pos_tags[l]][i - 2][0]

                        p1 = transition_probability[pos_tags[l]][pos_tags[j]][pos_tags[k]]
                        p2 = emission_probability[words[cursen[i * 2]]][pos_tags[k]]

                        cur_prob = prev_prob * prev_prev_prob * p1 * p2

                        if cur_prob >= max_prob:
                            max_prob = cur_prob
                            prob_and_seq[pos_tags[k]][i][0] = max_prob
                            prob_and_seq[pos_tags[k]][i][1] = j

        else:
            for k in pos_tags.keys():  # current state
                max_prob = float('-inf')
                max_postag = ""

                for j in pos_tags.keys():  # previous state
                    for l in pos_tags.keys():  # previous previous state
                        if k == most_frequent_tag:
                            prev_prob = prob_and_seq[pos_tags[j]][i - 1][0]
                            prev_prev_prob = prob_and_seq[pos_tags[l]][i - 2][0]

                            p1 = transition_probability[pos_tags[l]][pos_tags[j]][pos_tags[k]]

                            cur_prob = prev_prob * prev_prev_prob * p1

                            if cur_prob >= max_prob:
                                max_prob = cur_prob
                                prob_and_seq[pos_tags[k]][i][0] = max_prob
                                prob_and_seq[pos_tags[k]][i][1] = j
                        else:
                            prob_and_seq[pos_tags[k]][i][0] = float(0)
                            prob_and_seq[pos_tags[k]][i][1] = "O"

    # Backtrack to find the best sequence
    max_prob = float('-inf')
    prev_postag = ""

    for i in pos_tags.keys():
        if prob_and_seq[pos_tags[i]][cursenlen - 1][0] >= max_prob:
            max_prob = prob_and_seq[pos_tags[i]][cursenlen - 1][0]
            prev_postag = prob_and_seq[pos_tags[i]][cursenlen - 1][1]
            maxipostag = i

    predicted_seq_of_pos_tags.append(maxipostag)

    for i in range(cursenlen - 2, 0, -1):
        predicted_seq_of_pos_tags.append(prev_postag)
        prev_postag = prob_and_seq[pos_tags[prev_postag]][i][1]

    predicted_seq_of_pos_tags.reverse()

    return predicted_seq_of_pos_tags

In [ ]:
def test_bigram(X_test,transition_probability_t1_t2,emission_probability,start_probability,count_of_each_BIO_tags,BIO_tags,words,most_frequent_tag):
    predicted_y_bigram = []
    for i in X_test:
        predicted_y_bigram.append(viterbiAlgorithm_Bigram(transition_probability_t1_t2,emission_probability,start_probability,count_of_each_BIO_tags,BIO_tags,words,most_frequent_tag,i))
    return predicted_y_bigram

In [ ]:
def test_trigram(X_test, transition_probability_t1_t2_t3, emission_probability, start_probability, count_of_each_BIO_tags, BIO_tags, words, most_frequent_tag):
    predicted_y_trigram = []
    for i in X_test:
        predicted_y_trigram.append(viterbiAlgorithm_Trigram(transition_probability_t1_t2_t3, emission_probability, start_probability, count_of_each_BIO_tags, BIO_tags, words, most_frequent_tag, i))
    return predicted_y_trigram

In [ ]:
start = int(len(df)*0.8)
X_train = df.iloc[:start,0]
X_test = df.iloc[start:,0]
BIO_tags = {'B':0,'I':1,'O':2}

In [ ]:
def calculate_metrics(predicted_y):
    k = 0
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for i in X_test:
        count = 0
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        actualtags_y = i.split(" ")
        curlen = int(len(actualtags_y)/2)

        for j in range(0, curlen):
            if predicted_y[k][j] == actualtags_y[j*2 + 1]:
                count += 1
                true_positives += 1
            else:
                false_positives += 1
                false_negatives += 1

        k += 1

        # Calculate precision
        if true_positives + false_positives != 0:
            precision.append(true_positives / (true_positives + false_positives))
        else:
            precision.append(0)

        # Calculate recall
        if true_positives + false_negatives != 0:
            recall.append(true_positives / (true_positives + false_negatives))
        else:
            recall.append(0)

        # Calculate F1 score
        if precision[-1] + recall[-1] != 0:
            f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))
        else:
            f1.append(0)

        accuracy.append(count / curlen)

    avg_accuracy = sum(accuracy) / len(accuracy)
    avg_precision = sum(precision) / len(precision)
    avg_recall = sum(recall) / len(recall)
    avg_f1 = sum(f1) / len(f1)

    return avg_accuracy, avg_precision, avg_recall, avg_f1


# Example usage:
transition_probability_t1_t2, count_of_each_BIO_tags, BIO_tags_as_initial_count = calculate_transition_probability_bigram(X_train)
start_probability = calculate_start_probability(BIO_tags_as_initial_count, count_of_each_BIO_tags)
emission_probability, words = calculate_emission_probability(X_train, count_of_each_BIO_tags)
most_frequent_tag = most_frequent_BIO_tags(count_of_each_BIO_tags)

# Running bigram HMM Model
predicted_y_bi = test_bigram(X_test, transition_probability_t1_t2, emission_probability, start_probability, count_of_each_BIO_tags, BIO_tags, words, most_frequent_tag)

# Calculating metrics
accuracy, precision, recall, f1_score = calculate_metrics(predicted_y_bi)

print("Accuracy of bigram model at Fold = ", round(accuracy * 100, 2), '%', sep="")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Accuracy of bigram model at Fold = 93.74%
Precision: 0.9373917296363601
Recall: 0.9373917296363601
F1 Score: 0.9373917296363601


In [ ]:
def calculate_accuracy_trigram(predicted_y, X_test):
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for idx, i in enumerate(X_test):
        count = 0
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        actualtags_y = i.split(" ")
        curlen = len(actualtags_y) // 2

        if curlen <= 2:
            continue  # Skip calculation for sequences with length less than or equal to 2

        for j in range(1, curlen - 1):  # Trigram prediction starts from index 1
            if predicted_y[idx][j] == actualtags_y[j * 2 + 1]:
                count += 1
                true_positives += 1
            else:
                false_positives += 1
                false_negatives += 1

        # Calculate precision
        if true_positives + false_positives != 0:
            precision.append(true_positives / (true_positives + false_positives))
        else:
            precision.append(0)

        # Calculate recall
        if true_positives + false_negatives != 0:
            recall.append(true_positives / (true_positives + false_negatives))
        else:
            recall.append(0)

        # Calculate F1 score
        if precision[-1] + recall[-1] != 0:
            f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))
        else:
            f1.append(0)

        accuracy.append(count / (curlen - 2))  # Subtract 2 for ignoring start and end symbols

    avg_accuracy = sum(accuracy) / len(accuracy) if accuracy else 0  # Avoid division by zero
    avg_precision = sum(precision) / len(precision) if precision else 0
    avg_recall = sum(recall) / len(recall) if recall else 0
    avg_f1 = sum(f1) / len(f1) if f1 else 0

    return avg_accuracy, avg_precision, avg_recall, avg_f1


# Calculating parameters for trigram
transition_probability_t1_t2_t3, count_of_each_BIO_tags, BIO_tags_as_initial_count = calculate_transition_probability_trigram(X_train)
start_probability = calculate_start_probability(BIO_tags_as_initial_count, count_of_each_BIO_tags)
emission_probability, words = calculate_emission_probability(X_train, count_of_each_BIO_tags)
most_frequent_tag = most_frequent_BIO_tags(count_of_each_BIO_tags)
# Running trigram HMM Model
predicted_y_tri = test_trigram(X_test, transition_probability_t1_t2_t3, emission_probability, start_probability, count_of_each_BIO_tags, BIO_tags, words, most_frequent_tag)
accuracy, precision, recall, f1_score = calculate_accuracy_trigram(predicted_y_tri, X_test)
print("Accuracy of trigram model", round(accuracy * 100, 2), '%')
print("Precision of trigram model", round(precision * 100, 2), '%')
print("Recall of trigram model", round(recall * 100, 2), '%')
print("F1 Score of trigram model", round(f1_score * 100, 2), '%')

Accuracy of trigram model 93.95 %
Precision of trigram model 93.95 %
Recall of trigram model 93.95 %
F1 Score of trigram model 93.95 %


In [ ]:
total_count = len(df)
fold = 5
split_ratio = round(1 / fold)

avg_accuracy_bi = 0
avg_accuracy_tri = 0

avg_precision_bi = 0
avg_precision_tri = 0

avg_recall_bi = 0
avg_recall_tri = 0

avg_f1_bi = 0
avg_f1_tri = 0
BIO_tags = {'B': 0, 'I': 1, 'O': 2}

for i in range(fold):
    start = round(total_count / fold) * i
    end = round(total_count / fold) * (i + 1)

    X_train = df.iloc[np.r_[0:start, end:], -1].values
    X_test = df.iloc[start:end, -1].values



    # calculating parameters for bigram
    transition_probability_t1_t2, count_of_each_BIO_tags, BIO_tags_as_initial_count = calculate_transition_probability_bigram(X_train)
    start_probability = calculate_start_probability(BIO_tags_as_initial_count, count_of_each_BIO_tags)
    emission_probability, words = calculate_emission_probability(X_train, count_of_each_BIO_tags)
    most_frequent_tag = most_frequent_BIO_tags(count_of_each_BIO_tags)

    # running bigram HMM Model
    predicted_y_bi = test_bigram(X_test, transition_probability_t1_t2, emission_probability, start_probability, count_of_each_BIO_tags, BIO_tags, words, most_frequent_tag)
    accuracy, precision, recall, f1_score = calculate_metrics(predicted_y_bi)
    avg_accuracy_bi += accuracy
    avg_precision_bi += precision
    avg_recall_bi += recall
    avg_f1_bi += f1_score
    print("Bigram Model at Fold ", i, ":")
    print("Accuracy =", round(accuracy * 100, 2), '%')
    print("Precision =", round(precision * 100, 2), '%')
    print("Recall =", round(recall * 100, 2), '%')
    print("F1 Score =", round(f1_score * 100, 2), '%')




avg_accuracy_bi /= fold
avg_precision_bi /= fold
avg_recall_bi /= fold
avg_f1_bi /= fold



print("Average Accuracy of Bigram Model =", round(avg_accuracy_bi * 100, 2), '%')
print("Average Precision of Bigram Model =", round(avg_precision_bi * 100, 2), '%')
print("Average Recall of Bigram Model =", round(avg_recall_bi * 100, 2), '%')
print("Average F1 Score of Bigram Model =", round(avg_f1_bi * 100, 2), '%')

Bigram Model at Fold  0 :
Accuracy = 93.84 %
Precision = 93.84 %
Recall = 93.84 %
F1 Score = 93.84 %
Bigram Model at Fold  1 :
Accuracy = 93.3 %
Precision = 93.3 %
Recall = 93.3 %
F1 Score = 93.3 %
Bigram Model at Fold  2 :
Accuracy = 93.46 %
Precision = 93.46 %
Recall = 93.46 %
F1 Score = 93.46 %
Bigram Model at Fold  3 :
Accuracy = 95.41 %
Precision = 95.41 %
Recall = 95.41 %
F1 Score = 95.41 %
Bigram Model at Fold  4 :
Accuracy = 94.29 %
Precision = 94.29 %
Recall = 94.29 %
F1 Score = 94.29 %
Average Accuracy of Bigram Model = 94.06 %
Average Precision of Bigram Model = 94.06 %
Average Recall of Bigram Model = 94.06 %
Average F1 Score of Bigram Model = 94.06 %


In [ ]:
for i in range(fold):
    start = round(total_count / fold) * i
    end = round(total_count / fold) * (i + 1)

    X_train = df.iloc[np.r_[0:start, end:], -1].values
    X_test = df.iloc[start:end, -1].values


    # Calculating parameters for trigram
    transition_probability_t1_t2_t3, count_of_each_BIO_tags, BIO_tags_as_initial_count = calculate_transition_probability_trigram(X_train)
    start_probability = calculate_start_probability(BIO_tags_as_initial_count, count_of_each_BIO_tags)
    emission_probability, words = calculate_emission_probability(X_train, count_of_each_BIO_tags)
    most_frequent_tag = most_frequent_BIO_tags(count_of_each_BIO_tags)

    # Running trigram HMM Model
    predicted_y_tri = test_trigram(X_test, transition_probability_t1_t2_t3, emission_probability, start_probability, count_of_each_BIO_tags, BIO_tags, words, most_frequent_tag)
    accuracy, precision, recall, f1_score = calculate_accuracy_trigram(predicted_y_tri, X_test)
    avg_accuracy_tri += accuracy
    avg_precision_tri += precision
    avg_recall_tri += recall
    avg_f1_tri += f1_score
    print("Trigram Model at Fold ", i, ":")
    print("Accuracy =", round(accuracy * 100, 2), '%')
    print("Precision =", round(precision * 100, 2), '%')
    print("Recall =", round(recall * 100, 2), '%')
    print("F1 Score =", round(f1_score * 100, 2), '%')

avg_accuracy_tri /= fold
avg_precision_tri /= fold
avg_recall_tri /= fold
avg_f1_tri /= fold

print("Average Accuracy of Trigram Model =", round(avg_accuracy_tri * 100, 2), '%')
print("Average Precision of Trigram Model =", round(avg_precision_tri * 100, 2), '%')
print("Average Recall of Trigram Model =", round(avg_recall_tri * 100, 2), '%')
print("Average F1 Score of Trigram Model =", round(avg_f1_tri * 100, 2), '%')

Trigram Model at Fold  0 :
Accuracy = 93.31 %
Precision = 93.31 %
Recall = 93.31 %
F1 Score = 93.31 %
Trigram Model at Fold  1 :
Accuracy = 93.12 %
Precision = 93.12 %
Recall = 93.12 %
F1 Score = 93.12 %
Trigram Model at Fold  2 :
Accuracy = 93.27 %
Precision = 93.27 %
Recall = 93.27 %
F1 Score = 93.27 %
Trigram Model at Fold  3 :
Accuracy = 95.24 %
Precision = 95.24 %
Recall = 95.24 %
F1 Score = 95.24 %
Trigram Model at Fold  4 :
Accuracy = 93.95 %
Precision = 93.95 %
Recall = 93.95 %
F1 Score = 93.95 %
Average Accuracy of Trigram Model = 93.78 %
Average Precision of Trigram Model = 93.78 %
Average Recall of Trigram Model = 93.78 %
Average F1 Score of Trigram Model = 93.78 %
